In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
df1 = pd.read_excel('./data/시세추이.xlsx')

C:\Users\me\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# ATR(Average True Range) 변동성 지표 생성

df_atr = df1.iloc[:,[0,1,5,6]]
df_atr.columns = ['date', 'fin', 'high', 'low']

In [4]:
df_atr

,date,fin,high,low
0,2021/08/31,76700.0,76700.0,74300.0
1,2021/08/30,74600.0,75500.0,74200.0
2,2021/08/27,74300.0,75000.0,73800.0
3,2021/08/26,74600.0,76200.0,74600.0
4,2021/08/25,75700.0,76600.0,74900.0
...,...,...,...,...
2460,2011/09/06,727000.0,736000.0,710000.0
2461,2011/09/05,731000.0,756000.0,730000.0
2462,2011/09/02,769000.0,781000.0,763000.0
2463,2011/09/01,771000.0,788000.0,744000.0


In [5]:
lst = []
atr = []

for i in range(0,len(df_atr)-1):
    a = df_atr.iloc[i,2] - df_atr.iloc[i,3]
    b = df_atr.iloc[i,2] - df_atr.iloc[i+1,1]
    c = df_atr.iloc[i,3] - df_atr.iloc[i+1,1]
    lst = [abs(a),abs(b),abs(c)]
    atr.append(max(lst))

In [6]:
# y값(종가 데이터) 준비

fin = list(df_atr[0:-1]['fin'])
del fin[820:823] # 거래정지 시점 데이터 행 삭제

In [7]:
# 거래량 데이터 추출

df1 = df1.drop(len(df1)-1,axis=0)
df1 = df1[['일자', '거래량']]

In [8]:
# PER, PBR 추출

df2 = pd.read_excel('./data/PERPBR.xlsx')
df2 = df2[['일자', 'PER', 'PBR']]

In [9]:
# 거래주체별 순매수 데이터 추출

df3 = pd.read_excel('./data/거래실적.xlsx')
df3 = df3.drop(columns=['전체'], axis=1)

C:\Users\me\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
# 데이터 병합
samsung = pd.merge(df1, df2, on='일자', how='outer')
samsung = pd.merge(samsung, df3, on='일자', how='outer')
samsung['ATR'] = atr

In [11]:
# 거래정지 시점 데이터 행 삭제

samsung = samsung.drop(index = [820,821,822], axis=1)

In [12]:
# '2018/04/27' 액면분할에 따른 전처리

samsung.iloc[820:,8] = samsung.iloc[820:,8]/50
samsung.iloc[820:,1] = samsung.iloc[820:,1]*50

In [13]:
# 날짜 데이터로 변환

samsung['일자'] = pd.to_datetime(samsung['일자'])
samsung.rename(columns={'일자':'DATE'}, inplace=True)

In [14]:
# 거시경제 지표와 병합

macro_data = pd.read_csv('./data/merge_data.csv')
macro_data['DATE'] = pd.to_datetime(macro_data['DATE'])    # 날짜 데이터로 변환
macro_data = macro_data.dropna(axis=0)                     # 결측치가 포함된 행 제거
df_samsung = pd.merge(samsung, macro_data, how='left', on='DATE')

In [15]:
df_samsung

,DATE,거래량,PER,PBR,기관 합계,기타법인,개인,외국인 합계,ATR,NASDAQ,S&P,CBOE,Exchange rate,futures2y,futures10y
0,2021-08-31,24630370.0,19.97,1.95,-1.530797e+11,7.948321e+09,-2.149358e+11,3.600673e+11,2400.0,"15,259.20","4,522.68",16.48,"1,158.73",110.32,134.05
1,2021-08-30,12686999.0,19.42,1.89,2.458642e+10,-2.401179e+09,5.471519e+10,-7.690043e+10,1300.0,"15,265.90","4,528.79",16.19,"1,164.34",110.32,134.13
2,2021-08-27,15172748.0,19.34,1.89,1.731184e+11,1.037227e+10,1.510888e+09,-1.850016e+11,1200.0,"15,129.50","4,509.37",16.39,"1,161.23",110.30,133.89
3,2021-08-26,16671494.0,19.42,1.89,-5.556654e+10,9.344982e+09,1.425748e+11,-9.635320e+10,1600.0,"14,945.80","4,469.91",18.84,"1,170.53",110.25,133.59
4,2021-08-25,22319664.0,19.71,1.92,-7.699047e+10,-8.894668e+09,8.237323e+10,3.511909e+09,1700.0,"15,041.90","4,496.19",16.79,"1,163.87",110.25,133.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,2011-09-07,17575200.0,7.29,1.24,1.132324e+11,1.123519e+09,-7.790727e+10,-3.644860e+10,1020.0,"2,548.90","1,198.62",33.38,"1,069.65",110.27,130.89
2457,2011-09-06,13225250.0,6.86,1.17,2.349662e+09,-1.299409e+09,1.383982e+10,-1.489007e+10,520.0,"2,473.80","1,165.24",37.00,"1,074.65",110.27,131.22
2458,2011-09-05,13874400.0,6.90,1.17,-9.756990e+08,-2.156410e+08,2.937076e+10,-2.817942e+10,780.0,NaN,NaN,NaN,NaN,NaN,NaN
2459,2011-09-02,12934100.0,7.26,1.23,-7.185878e+09,2.125969e+09,-1.137706e+10,1.643697e+10,360.0,"2,480.30","1,173.97",33.92,"1,065.60",110.25,131.30


In [16]:
# y값(종가 데이터) 병합

df_samsung['y'] = fin

In [17]:
# 종가 열 액면분할 반영

df_samsung.iloc[820:,15] = df_samsung.iloc[820:,15]/50

In [18]:
# 결측치를 전일 값으로 대체

df_samsung.sort_values('DATE', inplace=True)
df_samsung.fillna(method='ffill', inplace=True)

In [19]:
df_samsung.to_excel('./data/samsung.xlsx',index=False)

In [20]:
samsung_df = pd.read_excel('./data/samsung.xlsx')

In [21]:
samsung_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DATE           2461 non-null   datetime64[ns]
 1   거래량            2461 non-null   int64         
 2   PER            2461 non-null   float64       
 3   PBR            2461 non-null   float64       
 4   기관 합계          2461 non-null   int64         
 5   기타법인           2461 non-null   int64         
 6   개인             2461 non-null   int64         
 7   외국인 합계         2461 non-null   int64         
 8   ATR            2461 non-null   int64         
 9   NASDAQ         2461 non-null   object        
 10  S&P            2461 non-null   object        
 11  CBOE           2461 non-null   float64       
 12  Exchange rate  2461 non-null   object        
 13  futures2y      2461 non-null   float64       
 14  futures10y     2461 non-null   float64       
 15  y              2461 n